In [ ]:
import pandas as pd
import numpy as np

# Định nghĩa tên cột
column_names = ["Id", "Name", "Age", "Weight", "m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]

# Đọc file CSV và thêm header
file_path = "patient_heart_rate.csv"
df = pd.read_csv(file_path, names=column_names)

# Tách cột Name thành Firstname và Lastname
df[['Firstname', 'Lastname']] = df['Name'].str.split(expand=True)
df.drop(columns=['Name'], inplace=True)

# Chuẩn hóa đơn vị cột Weight (giả sử có lbs chuyển sang kg)
def convert_weight(value):
    if isinstance(value, str) and "lbs" in value:
        return round(float(value.replace("lbs", "")) * 0.453592, 2)
    return float(value)
df['Weight'] = df['Weight'].apply(convert_weight)

# Xóa dòng trống
df.dropna(how='all', inplace=True)

# Loại bỏ dòng trùng lặp
df.drop_duplicates(inplace=True)

# Xử lý ký tự không phải ASCII
def remove_non_ascii(text):
    if isinstance(text, str):
        return text.encode("ascii", "ignore").decode()
    return text
df = df.applymap(remove_non_ascii)

# Xử lý giá trị thiếu của Age và Weight
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Weight'].fillna(df['Weight'].mean(), inplace=True)
df.dropna(subset=['Age', 'Weight'], how='all', inplace=True)

# Tách dữ liệu từ các cột huyết áp
def process_pulse_rate(value):
    if pd.isna(value):
        return np.nan
    return int(value)
df[['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']] = df[['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']].applymap(process_pulse_rate)

# Xử lý giá trị thiếu trong các cột huyết áp
heart_rate_columns = ['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']
for col in heart_rate_columns:
    df[col] = df[col].fillna(df[col].interpolate(method='linear', limit_direction='both'))
    df[col].fillna(df[col].mean(), inplace=True)

# Lưu dữ liệu đã xử lý vào file mới
df.to_csv("patient_heart_rate_clean.csv", index=False)

print("Data cleaning complete. Processed file saved as 'patient_heart_rate_clean.csv'.")
